In [1]:
import pandas as pd
import numpy as np
import requests, time
from bs4 import BeautifulSoup
import sqlalchemy

In [2]:
url = 'https://www.pro-football-reference.com/years/2022/passing.htm'

In [3]:
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html)
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/pfr/build" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <link href="https://cdn.ssref.net/req/202301251" rel="dns-prefetch"/>
  <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
  <script async="true" type="text/javascript">
   (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME =

In [4]:
# Collect table headers
column_headers = soup.findAll('tr')[0]
column_headers = [i.getText() for i in column_headers.findAll('th')]

In [5]:
print(column_headers)

['Rk', 'Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'Sk%', 'NY/A', 'ANY/A', '4QC', 'GWD']


In [6]:
# Collect table rows
rows = soup.find_all('tr')[1:]

#get stats from each row
qb_stats = []
for i in range(len(rows)):
    qb_stats.append( [col.getText() for col in rows[i].find_all('td')])    

In [7]:
print(qb_stats[0])

['Patrick Mahomes*+', 'KAN', '27', 'QB', '17', '17', '14-3-0', '435', '648', '67.1', '5250', '41', '6.3', '12', '1.9', '272', '67', '8.1', '8.5', '12.1', '308.8', '105.2', '77.6', '26', '188', '3.9', '7.51', '7.93', '4', '4']


In [8]:
#Create DatFrame
data = pd.DataFrame(qb_stats, columns=column_headers[1:])
data

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,Sk%,NY/A,ANY/A,4QC,GWD
0,Patrick Mahomes*+,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins*,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow*,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend*+,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [9]:
#Remove value None
data = data.dropna()

In [10]:
data.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%',
       'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C',
       'Y/G', 'Rate', 'QBR', 'Sk', 'Yds', 'Sk%', 'NY/A', 'ANY/A', '4QC',
       'GWD'],
      dtype='object')

In [11]:
#Duas colunas com mesmo nome (Yds), pois uma é jardas ganhas e a outra é 
#jardas perdidas devido a bloqueios/detidos
#Renomear a coluna jardas detidas
new_column = data.columns.values
new_column[-6] = 'yds_sk'
data.columns = new_column

data.columns

Index(['Player', 'Tm', 'Age', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att', 'Cmp%',
       'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Lng', 'Y/A', 'AY/A', 'Y/C',
       'Y/G', 'Rate', 'QBR', 'Sk', 'yds_sk', 'Sk%', 'NY/A', 'ANY/A', '4QC',
       'GWD'],
      dtype='object')

In [12]:
data

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,yds_sk,Sk%,NY/A,ANY/A,4QC,GWD
0,Patrick Mahomes*+,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins*,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow*,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend*+,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [13]:
data['Player'] = data['Player'].str.replace('*', '')

C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\2318361915.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Player'] = data['Player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\2318361915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Player'] = data['Player'].str.replace('*', '')


In [15]:
data['Player'] = data['Player'].str.replace('+', '')

C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\769207374.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Player'] = data['Player'].str.replace('+', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\769207374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Player'] = data['Player'].str.replace('+', '')


In [16]:
data

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,yds_sk,Sk%,NY/A,ANY/A,4QC,GWD
0,Patrick Mahomes,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [17]:
def get_data(url):
    response = requests.get(url)
    html = response.content
    

def get_headers(html):
    headers = soup.findAll('tr')[0]
    headers = [i.getText() for i in headers.findAll('th')]
    
    
def setup(url):
    pure_data = get_data(url)
    soup = BeautifulSoup(pure_data)
    col_headers = get_headers(soup)
    rows = soup.find_all('tr')[1:]
    
def populate_qb(rows):
    qb_stats = []
    for i in range(len(rows)):
        qb_stats.append( [col.getText() for col in rows[i].find_all('td')])
        
    return qb_stats
    
def init_dataframe(qb_stats, column_headers):    
    data = pd.DataFrame(qb_stats, columns=column_headers[1:])
    
    return data

def sanitizer_qb():
    data = data.dropna()
    new_column = data.columns.values
    new_column[-6] = 'yds_sk'
    data.columns = new_column
    data.columns = [column.lower().strip() for column in data.columns]
    data['player'] = data['player'].str.replace('*', '')
    data['player'] = data['player'].str.replace('+', '')
    
    return data
    

In [18]:
def get_data(url):
    response = requests.get(url)
    html = response.content
    
    return html

In [19]:
get_data(url)

b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/pfr/build" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://cdn.ssref.net/req/202301251" />\n<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->\n<script type="text/javascript" async=true>\n    (function() {\n\tvar host = window.location.hostname;\n\tvar element = document.createElement(\'script\');\n\tvar firstScript = document.getElementsByTagName(\'script\')[0];\n\tvar url = \'https://cmp.quantcast.com\'\n\t    .concat(\'/choice/\', \'XwNYEpNeFfhfr\', \'/\', host, \n\t\t    \'/choice.js?tag_version=V2\');\n\tvar uspTries = 0;\n\tvar uspTriesLimit = 3;\n\telement.async = true;\n\telement.type = \'text/javascript\';\n\telement.src = url;\n\t\n\tfirstScript.parentNode.insertBefore(element, firstSc

In [20]:
def get_headers(soup):
    headers = soup.findAll('tr')[0]
    headers = [i.getText() for i in headers.findAll('th')]
    
    return headers

In [21]:
get_headers(soup)

['Rk',
 'Player',
 'Tm',
 'Age',
 'Pos',
 'G',
 'GS',
 'QBrec',
 'Cmp',
 'Att',
 'Cmp%',
 'Yds',
 'TD',
 'TD%',
 'Int',
 'Int%',
 '1D',
 'Lng',
 'Y/A',
 'AY/A',
 'Y/C',
 'Y/G',
 'Rate',
 'QBR',
 'Sk',
 'Yds',
 'Sk%',
 'NY/A',
 'ANY/A',
 '4QC',
 'GWD']

In [171]:
def setup(url):
    pure_data = get_data(url)
    soup = BeautifulSoup(pure_data)
    col_headers = get_headers(soup)
    rows = soup.find_all('tr')[1:]
    
    return rows

In [172]:
setup(url)

[<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Mahomes,Patrick" data-append-csv="MahoPa00" data-stat="player"><a href="/players/M/MahoPa00.htm">Patrick Mahomes</a>*+</td><td class="left" data-stat="team"><a href="/teams/kan/2022.htm" title="Kansas City Chiefs">KAN</a></td><td class="right" data-stat="age">27</td><td class="left" data-stat="pos">QB</td><td class="right" data-stat="g">17</td><td class="right" data-stat="gs">17</td><td class="right" csk="0.82352941176" data-stat="qb_rec">14-3-0</td><td class="right" data-stat="pass_cmp">435</td><td class="right" data-stat="pass_att">648</td><td class="right" data-stat="pass_cmp_perc">67.1</td><td class="right" data-stat="pass_yds">5250</td><td class="right" data-stat="pass_td">41</td><td class="right" data-stat="pass_td_perc">6.3</td><td class="right" data-stat="pass_int">12</td><td class="right" data-stat="pass_int_perc">1.9</td><td class="right" data-stat="pass_first_down">272</td><td class="ri

In [22]:
def populate_qb(rows):
    qb_stats = []
    for i in range(len(rows)):
        qb_stats.append( [col.getText() for col in rows[i].find_all('td')])
        
    return qb_stats

In [23]:
populate_qb(rows)

[['Patrick Mahomes*+',
  'KAN',
  '27',
  'QB',
  '17',
  '17',
  '14-3-0',
  '435',
  '648',
  '67.1',
  '5250',
  '41',
  '6.3',
  '12',
  '1.9',
  '272',
  '67',
  '8.1',
  '8.5',
  '12.1',
  '308.8',
  '105.2',
  '77.6',
  '26',
  '188',
  '3.9',
  '7.51',
  '7.93',
  '4',
  '4'],
 ['Justin Herbert',
  'LAC',
  '24',
  'QB',
  '17',
  '17',
  '10-7-0',
  '477',
  '699',
  '68.2',
  '4739',
  '25',
  '3.6',
  '10',
  '1.4',
  '228',
  '55',
  '6.8',
  '6.9',
  '9.9',
  '278.8',
  '93.2',
  '58.3',
  '38',
  '206',
  '5.2',
  '6.15',
  '6.22',
  '4',
  '5'],
 ['Tom Brady',
  'TAM',
  '45',
  'QB',
  '17',
  '17',
  '8-9-0',
  '490',
  '733',
  '66.8',
  '4694',
  '25',
  '3.4',
  '9',
  '1.2',
  '237',
  '63',
  '6.4',
  '6.5',
  '9.6',
  '276.1',
  '90.7',
  '52.5',
  '22',
  '160',
  '2.9',
  '6.01',
  '6.13',
  '4',
  '5'],
 ['Kirk Cousins*',
  'MIN',
  '34',
  'QB',
  '17',
  '17',
  '13-4-0',
  '424',
  '643',
  '65.9',
  '4547',
  '29',
  '4.5',
  '14',
  '2.2',
  '230',
  '66'

In [24]:
def init_dataframe(qb_stats, column_headers):    
    data = pd.DataFrame(qb_stats, columns=column_headers[1:])
    
    return data

In [25]:
init_dataframe(qb_stats, column_headers)

,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,Cmp%,...,Y/G,Rate,QBR,Sk,Yds,Sk%,NY/A,ANY/A,4QC,GWD
0,Patrick Mahomes*+,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins*,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow*,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend*+,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [26]:
def sanitizer_qb(data):
    data = data.dropna()
    new_column = data.columns.values
    new_column[-6] = 'yds_sk'
    data.columns = new_column
    data.columns = [column.lower().strip() for column in data.columns]
    data['player'] = data['player'].str.replace('*', '')
    data['player'] = data['player'].str.replace('+', '')
    
    return data

In [27]:
sanitizer_qb(data)

C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['player'] = data['player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['player'] = data['player'].str.replace('+', '')


,player,tm,age,pos,g,gs,qbrec,cmp,att,cmp%,...,y/g,rate,qbr,sk,yds_sk,sk%,ny/a,any/a,4qc,gwd
0,Patrick Mahomes,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [28]:
def robos(url):
    page_html = get_data(url)
    soup = BeautifulSoup(page_html)
    name_cols = get_headers(soup)
    contents_row = soup.find_all('tr')[1:]
    traits_qb = populate_qb(contents_row)
    
    table = init_dataframe(traits_qb, name_cols)
    return sanitizer_qb(table)
    
    
    

In [29]:
robos(url)

C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['player'] = data['player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player'] = data['player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=Tru

,player,tm,age,pos,g,gs,qbrec,cmp,att,cmp%,...,y/g,rate,qbr,sk,yds_sk,sk%,ny/a,any/a,4qc,gwd
0,Patrick Mahomes,KAN,27,QB,17,17,14-3-0,435,648,67.1,...,308.8,105.2,77.6,26,188,3.9,7.51,7.93,4,4
1,Justin Herbert,LAC,24,QB,17,17,10-7-0,477,699,68.2,...,278.8,93.2,58.3,38,206,5.2,6.15,6.22,4,5
2,Tom Brady,TAM,45,QB,17,17,8-9-0,490,733,66.8,...,276.1,90.7,52.5,22,160,2.9,6.01,6.13,4,5
3,Kirk Cousins,MIN,34,QB,17,17,13-4-0,424,643,65.9,...,267.5,92.5,49.9,46,329,6.7,6.12,6.05,8,8
4,Joe Burrow,CIN,26,QB,16,16,12-4-0,414,606,68.3,...,279.7,100.8,58.7,41,259,6.3,6.52,6.76,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Christian Kirk,JAX,26,WR,17,17,,0,1,0.0,...,0.0,39.6,9.8,0,0,0.0,0.00,0.00,,
105,Cooper Kupp,LAR,29,WR,9,9,,0,1,0.0,...,0.0,39.6,3.0,0,0,0.0,0.00,0.00,,
106,James Proche,BAL,26,WR,15,0,,0,1,0.0,...,0.0,0.0,0.0,0,0,0.0,0.00,-45.00,,
107,Tommy Townsend,KAN,26,P,17,0,,0,1,0.0,...,0.0,39.6,,0,0,0.0,0.00,0.00,,


In [34]:
df = robos(url)


C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['player'] = data['player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player'] = data['player'].str.replace('*', '')
C:\Users\mateu\AppData\Local\Temp\ipykernel_14524\1972171126.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=Tru

In [31]:
import sqlalchemy
db_server='pymysql'
user='root'
db_port = '3306'
password = 'rootroot'
ip = 'localhost'
db_name = 'nflcopy'
engine = sqlalchemy.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')

In [47]:
filename = 'quarterback'+'.csv'
filename

'quarterback.csv'

In [48]:
df.to_csv(filename,index=False)

In [49]:
df = pd.read_csv(filename)
filename

'quarterback.csv'

In [50]:
table_name = filename.split('.')[0]
table_name

'quarterback'

In [51]:
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

106

In [54]:
sqlalchemy.inspect(engine).get_table_names()

['quarterback']